<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*This notebook contains an excerpt from the [Python Business Automation Handbook](https://smartai1004.tistory.com/46?category=946283) by Smart ICT; 
the content is available [on GitHub](https://github.com/aiml1004/automate_basic).*

*본 자료를 무단 복제, 배포하는 경우에는 저작권법 제97조의 저작재산권침해죄에 해당하여 저작권법에 의거 법적 조치에 처해질 수 있습니다.*
*COPYRIGHT ⓒ 2020 SMART ICT ALL RIGHTS RESERVED.* 

*If you find this content useful, please consider supporting the work by [buying the book](https://kmong.com/gig/233618)!*

<!--NAVIGATION-->
< [Preface](00.00-Preface.ipynb) | [Contents](00_Index.ipynb) | [Help and Documentation in IPython](01.01-Help-And-Documentation.ipynb) >

## 시나리오

무엇을 하고 싶은가?  

정보통신부 사업공고 게시판의 내용을 확인해 새로운 게시글이 있다면 공공영업팀장 및 팀원들께 메일을 보낸다.   
메일은 매일 아침 8시에 자동으로 보낸다. (월~금 weekday에만 수행)  

과학기술정보통신부에 접속한다.   
사업공고 게시판의 게시글을 엑셀로 저장한다.   
어제와 다른 새로운 게시글(사업공고)이 있는지 확인한다.   
새로운 게시글의 첨부문서를 다운로드한다.   
오늘 다운로드한 게시글을 엑셀로 저장한다.   

영업활동에 참고하시도록 영업팀장님께 메일로 보낸다.    
  오늘 다운로드한 게시글 엑셀   
  신규과제 공고문(신청요강 포함), 연구개발계획서(양식) 등 첨부파일은 사업별로 압축파일을 만든다.   

### 제약조건
MS 윈도우를 기준으로 작성 했습니다. 제가 맥 사용경험이 없습니다.  
다만 리눅스/유닉스에 경험이 있으니 유닉스를 기준으로 말씀드릴수는 있습니다.

In [ ]:
# https://cozynow.tistory.com/43

In [ ]:
# 02_사업공고_조회

In [49]:
import pandas as pd
from time import sleep
import os, sys
import shutil, glob, time

In [10]:
BASE_DIR = 'C:\\mylab\\book\\automate_basic'
BOARD_LIST_XL = BASE_DIR+'\\data\\board_list.xlsx'
SEND_HIST_XL  = BASE_DIR+'\\data\\mail_send_hsitory.xlsx'
NEW_POST_LIST_XL = BASE_DIR+'\\data\\new_post_list.xlsx'
DOWNLOAD_DIR = BASE_DIR+'\\download' #첨부문서 다운로드 위치. 우리가 원하는 곳에 저장하기 위한 설정

## import webdriver 

In [2]:
from selenium import webdriver
chromedriver = "C:\\mylab\\book\\chromedriver"

In [3]:
driver = webdriver.Chrome(chromedriver)

### 과학기술정보통신부 사업공고 게시판 HTML 분석

In [4]:
# 과학기술정보통신부 사업공고 게시판 
# 목적: 구조를 파악한다

[과학기술정보통신부 사업공고 게시판](https://www.msit.go.kr/web/msipContents/contents.do?mId=MTAzNQ==)

In [5]:
driver.get('https://www.msit.go.kr/web/msipContents/contents.do?mId=MTAzNQ==') # 이 URL은 어떻게 알게 되었는가?

## 게시판 게시글을 가져오려면?

In [ ]:
# 어떻게 우리가 원하는 항목을 가져올까요?
# 게시판은 어떤식으로 반복되고 있나요?

In [ ]:
# driver.find_elements_by_tag_name?

In [11]:
elem = driver.find_element_by_class_name('board_list') #왜 class_name 인가?

In [12]:
a_tag_list = elem.find_elements_by_tag_name('a') # Returns: - list of WebElement
# elements가 복수형이다

In [13]:
print("게시글 수는 " + str(len(a_tag_list)) +"개 입니다") 

게시글 수는 10개 입니다


In [14]:
a_tag_list[0].text # 첫번째 게시글 a_tag text,  
#파이썬 리스트인덱스는 0번째 부터 시작한다

'1074\n2020년도 EU 협력진흥사업 신규과제 공모\n부서 구주아프리카협력담당관 : 이세원 연락처 044-202-4354\n첨부파일 2020.09.29.'

In [15]:
a_tag_list[1].text

'1073\n2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모\n부서 구주아프리카협력담당관 : 이세원 연락처 044-202-4354\n첨부파일 2020.09.29.'

In [16]:
num= a_tag_list[0].find_element_by_class_name('num')
print('num: ', num.text)

title= a_tag_list[0].find_element_by_class_name('title')
print('title: ', title.text)

num:  1074
title:  2020년도 EU 협력진흥사업 신규과제 공모


### 부서와 연락처를 추출

In [17]:
# 문제점: class 이름이 con으로 동일하다

In [18]:
contact = a_tag_list[0].find_element_by_class_name('con') # 이것을 사용하면 안된다. 전화번호를 가져올수 없다.
contact.text

'구주아프리카협력담당관 : 이세원'

In [19]:
# elements가 복수형이다, con class를 모두 찾는다.
contact_lst= a_tag_list[0].find_elements_by_class_name('con')

In [20]:
# 0번째는 부서가 1번째는 전화번호가 저장되어 있다. 
contact_lst[0].text 

'구주아프리카협력담당관 : 이세원'

In [21]:
contact_lst[1].text

'044-202-4354'

In [22]:
date= a_tag_list[0].find_element_by_class_name('date') #element 단수형
date.text

'2020.09.29.'

In [23]:
date.text[0:10] # 마지막에 .이 있다. .을 제거한다.

'2020.09.29'

In [24]:
# 고유번호로 사용할 만한 것은 무엇인가? 사람의 경우는 주민번호와 같은
# 한개의 게시물을 고유하게 식별할 만한 것이 있는가? 
# 이것으로 어제 올라온 게시물과 비교해 새로운 게시물인지 아는데 활용한다
# artId가 고유번호인것 같다. 만약 저런 것이 없다면 사업명으로 구분할수밖에 없을 것같다.

In [25]:
u = a_tag_list[0].get_attribute('href') # 키보드 치기 싫어서 짧은 변수에 담는다. 어차피 지울 거라
u 

'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914'

In [26]:
find_str = "&artId="

print("u:", u)
print("")
print("""찾은 문자열 위치, u.find("&artId=") : """, u.find("&artId="))
print("문자열 길이:", len(u))
print("u[70:84] :", u[70:84])
print("")

print("len(find_str) : ", len(find_str))
print("u[70+7:84] : ", u[70+7:84])

print("-"*5)
print("최종 : ", u[u.find(find_str)+len(find_str):len(u)])

u: https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914

찾은 문자열 위치, u.find("&artId=") :  70
문자열 길이: 84
u[70:84] : &artId=3114914

len(find_str) :  7
u[70+7:84] :  3114914
-----
최종 :  3114914


In [27]:
rows = [] #게시글을 담을 리스트형 변수 
# a_tag_list에서 한줄씩 꺼낸다(to a_tag).
# 위에서 테스트한 문장을 정리한다.  a_tag_list[0] --> a_tag 로 변경

for a_tag in a_tag_list:   
    #print(a_tag.text)
    num= a_tag.find_element_by_class_name('num')
    title= a_tag.find_element_by_class_name('title')
    contact_lst= a_tag.find_elements_by_class_name('con')
    date= a_tag.find_element_by_class_name('date')
    link = a_tag.get_attribute('href')
    
    find_str = "&artId="
    post_id = link[link.find(find_str)+len(find_str):len(link)]
    row = (num.text, title.text, contact_lst[0].text, contact_lst[1].text, date.text[0:10], link, post_id)
    rows.append(row)

In [28]:
print(rows[0])
print(rows[1])

('1074', '2020년도 EU 협력진흥사업 신규과제 공모', '구주아프리카협력담당관 : 이세원', '044-202-4354', '2020.09.29', 'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114914', '3114914')
('1073', '2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모', '구주아프리카협력담당관 : 이세원', '044-202-4354', '2020.09.29', 'https://www.msit.go.kr/web/msipContents/contentsView.do?cateId=_tstb17&artId=3114913', '3114913')


In [29]:
columns = ['순번', '사업명', '공고_담당자', '전화번호', '공고일', 'URL', 'post_id']

In [30]:
df = pd.DataFrame(rows, columns=columns)

In [31]:
df

,순번,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,1074,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,1073,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,1072,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
3,1071,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
4,1070,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961
5,1069,2020년도 과학기술정보통신부 비대면 기업홍보 지원사업(언택트 IR) 통합공고,정보통신산업기반과 : 신유진,044-202-6245,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088727
6,1068,2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088726
7,1067,2020년도 재난안전플랫폼기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081711
8,1066,2020년도 국민생활안전 긴급대응연구사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081710
9,1065,2020년도 제2차 해외고급과학자초빙사업 Plus (Brain Pool+) 연장공고,미래인재양성과 : 양성녀,044-202-4836,2020.09.01,https://www.msit.go.kr/web/msipContents/conten...,3067835


In [32]:
df.to_excel(BOARD_LIST_XL, encoding='cp949', index=False)

## 특정 사업공고의 첨부파일을 다운로드 한다

### before

In [34]:
df = pd.read_excel(BOARD_LIST_XL)

In [35]:
df.head()

,순번,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,1074,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,1073,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,1072,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
3,1071,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
4,1070,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961


In [36]:
df

,순번,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,1074,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,1073,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913
2,1072,2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.28,https://www.msit.go.kr/web/msipContents/conten...,3122773
3,1071,2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.18,https://www.msit.go.kr/web/msipContents/conten...,3103448
4,1070,2020년도 기초연구실지원(기초연구 정책분야) 신규과제 공모,기초연구진흥과 : 오늘,044-202-4538,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088961
5,1069,2020년도 과학기술정보통신부 비대면 기업홍보 지원사업(언택트 IR) 통합공고,정보통신산업기반과 : 신유진,044-202-6245,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088727
6,1068,2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모,공공기술기반팀 : 허대성,044-202-4639,2020.09.10,https://www.msit.go.kr/web/msipContents/conten...,3088726
7,1067,2020년도 재난안전플랫폼기술개발사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081711
8,1066,2020년도 국민생활안전 긴급대응연구사업 신규과제 선정결과 공고,공공기술기반팀 : 허대성,044-202-4639,2020.09.07,https://www.msit.go.kr/web/msipContents/conten...,3081710
9,1065,2020년도 제2차 해외고급과학자초빙사업 Plus (Brain Pool+) 연장공고,미래인재양성과 : 양성녀,044-202-4836,2020.09.01,https://www.msit.go.kr/web/msipContents/conten...,3067835


In [ ]:
query_str = "사업명 in ['2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고', '2020년도 제5차 국민생활안전 긴급대응연구사업 신규과제 공모']"  #  (소문자 in 연산자)
#query_str = "사업명 == ['2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고', '2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모']" # 위와 동일한 표현식이다
df = df.query(query_str)     # 조건 부합 데이터 추출
df

In [ ]:
# df.itertuples? 
# Iterate over DataFrame rows as namedtuples.
# namedtuple이란?

In [ ]:
# df.itertuples?
for row in df.itertuples(index=True): #dataframe 칼럼명과 동일한 칼럼명을 가진 튜플을 리턴(반환)한다.
    print(row.URL)

In [ ]:
row

In [ ]:
# Index=3, 순번=1068, 사업명='2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모',
# dataframe의 칼럼명 = 값 형태로 매핑되어 있다.  칼럼명은 다양한 이름으로 불린다. attribute(속성), 변수(variable) 등

In [ ]:
row.URL

In [ ]:
driver.get(row.URL)

In [ ]:
elem = driver.find_element_by_class_name('down_file')

In [ ]:
a_tag_list = elem.find_elements_by_tag_name('a')

In [ ]:
a_tag = a_tag_list[0]

In [ ]:
a_tag.text

In [ ]:
# 아래 처럼, 링크가 아닌 자바스크립트가 들어가있는 경우에는 
# <a href = '#', onclick = 자바스크립트_명령어> 다운로드 </a>

# click() 명령을 하더라도  href 속성 값에 url 주소가 제대로 들어가있지 않기 때문에 해당 주소로 들어갈 수가 없습니다.
# 이런 경우에는, onlclick 내부의 명령어가 실행되도록 해야 합니다. 

In [ ]:
my_url = a_tag.get_attribute('href')
print(my_url)  # javascript:void(0);  원하는 url 아니다.

In [ ]:
a_tag_list[0].get_attribute('onclick') # 자바스크립트 문장을 가져온다. 

In [ ]:
a_tag_list[1].get_attribute('onclick') # 자바스크립트 문장을 가져온다. 

In [ ]:
# 우리가 원하는 문장은 1번(a_tag_list[1] )에 들어 있다.  첨부파일은 다운로드에 지정된 문장은 filePathEncode로 시작한다.

In [ ]:
for a_tag in a_tag_list: # 어떤 내용이 있는지 전부 출력해 보자.
    print(a_tag.get_attribute('onclick'))
    print("")

In [ ]:
#filePathEncode 만 출력해보자
for a_tag in a_tag_list:
    #print(a_tag.get_attribute('onclick'))
    txt = a_tag.get_attribute('onclick')
    if "filePathEncode" in txt:  
        print(txt)    

In [ ]:
# 첨부문서가 제대로 다운로드가 되는지 확인해 보자
js_script ="""filePathEncode('/cms/www/officialNews/biz/__icsFiles/afieldfile/2020/09/10/(붙임1) 2020년도 제4차 국민생활안전 긴급대응연구사업 신규과제 재공모 공고문.hwp')"""
driver.execute_script(js_script, a_tag)  #자바 명령어 실행
# sleep(1)

In [ ]:
# C:\Users\ME\Downloads 에 다운로드가 된다.

### refactoring_after

#### download default_directory 설정

In [37]:
from selenium.webdriver.chrome.options import Options

In [38]:
chrome_options = Options()

In [39]:
# chrome_options.add_experimental_option?

In [40]:
# Adds an experimental option which is passed to chrome.
prefs = {"download.default_directory" : DOWNLOAD_DIR} # prefs : to specify preferences that will be applied after Chrome starts
chrome_options.add_experimental_option("prefs",prefs)  
driver = webdriver.Chrome(executable_path=chromedriver, options=chrome_options)

## 신규 게시글의 첨부파일만 다운로드 한다

### before

In [ ]:
# 첨부 파일 저장 위치 테스트
date="2020\\09\\10"
biz_name="2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고"
outpath = DOWNLOAD_DIR+"\\{date1}\\{biz_name1}".format(date1=date, biz_name1=biz_name)
outpath

In [ ]:
DOWNLOAD_DIR+"\\*.*"

In [ ]:
# os.makedirs?

In [ ]:
# DOWNLOAD_DIR의 파일을 이동한다. 
os.makedirs(outpath) # https://docs.python.org/3/library/os.html 폴더를 만든다.
for file in glob.glob(DOWNLOAD_DIR+"\\*.*"):
    print(file)
    shutil.move(file, outpath)

In [ ]:
# C:\mylab\book\lecture\download\2020\09\18\2020년도 미래국방혁신기술개발사업 신규과제 선정결과 공고\

In [ ]:
outpath

In [ ]:
os.path.exists(outpath)

In [ ]:
row.사업명

In [ ]:
date = row.공고일.replace(".","//")
date

In [ ]:
# os.makedirs(outpath);

In [ ]:
df

### refactoring

In [41]:
# 다운로드 대상을 식별한다.
def get_send_df(new_xls_path, send_hist_xls_path):  #함수 정의   
    df = pd.read_excel(new_xls_path, sheet_name='Sheet1') #, index=False
    df_send_hist = pd.read_excel(send_hist_xls_path)     
    
    df_merge = df.merge(df_send_hist, how='left', indicator=True)
    df_diff = df_merge[df_merge['_merge'] == 'left_only']
    df_diff = df_diff.drop(columns=['_merge'])
    return  df_diff

In [42]:
# df_diff.drop?

In [43]:
df_diff = get_send_df(BOARD_LIST_XL, SEND_HIST_XL)

In [44]:
df_diff

,순번,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,1074,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,1073,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913


In [55]:
import my_util as ut
import time
def mkdir_rm(path0):
    if os.path.exists(path0):
        shutil.rmtree(path0)
    time.sleep(0.5)
    # os.mkdir(path0)
    os.makedirs(path0)

In [56]:
for row in df_diff.itertuples(index=True):
    #print(row.URL)
    driver.get(row.URL)
    sleep(2) # 게시판의 내용이 조회 완료되어야 함
    date = row.공고일.replace(".","\\")
    # Path to be created
    outpath = DOWNLOAD_DIR+"\\{date}\\{biz_name}".format(date=date, biz_name=row.사업명)
    
    elem = driver.find_element_by_class_name('down_file')
    a_tag_list = elem.find_elements_by_tag_name('a')  
    for a_tag in a_tag_list:        
        if "filePathEncode(" in a_tag.get_attribute('onclick'): 
            driver.execute_script(a_tag.get_attribute('onclick'), a_tag)  #자바스크립트 실행
            sleep(1)
            
    for file in glob.glob(DOWNLOAD_DIR+"\\*.*"):
        ut.mkdir_rm(outpath)
        shutil.move(file, outpath) #타겟(outpath)이 있으면 에러 발생

### refactoring 이전

In [ ]:
df = pd.read_excel(BOARD_LIST_XL, sheet_name='Sheet1') #, index=False
df.head()

In [ ]:
df_send_hist = pd.read_excel(SEND_HIST_XL) 
df_send_hist.head()

In [ ]:
# 다운로드 대상을 식별한다.
df_merge = df.merge(df_send_hist, how='left', indicator=True)  # indicator 칼럼 추가 > _merge

In [ ]:
df_merge.head()

In [ ]:
df_diff= df_merge[df_merge['_merge'] == 'left_only']
df_diff

## 메일을 보낸다

In [57]:
df_diff

,순번,사업명,공고_담당자,전화번호,공고일,URL,post_id
0,1074,2020년도 EU 협력진흥사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114914
1,1073,2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모,구주아프리카협력담당관 : 이세원,044-202-4354,2020.09.29,https://www.msit.go.kr/web/msipContents/conten...,3114913


In [58]:
from my_util import zip
for row in df_diff.itertuples(index=True):    
    date = row.공고일.replace(".","\\")
    download_full_path = DOWNLOAD_DIR+"\\{date}\\{biz_name}".format(date=date, biz_name=row.사업명)
    print(download_full_path)
    file = BASE_DIR+'\\send_temp\\{f_name}.zip'.format(f_name=row.사업명)
    print(file)    
    zip(download_full_path, file)    

C:\mylab\book\automate_basic\download\2020\09\29\2020년도 EU 협력진흥사업 신규과제 공모
C:\mylab\book\automate_basic\send_temp\2020년도 EU 협력진흥사업 신규과제 공모.zip
C:\mylab\book\automate_basic\download\2020\09\29\2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모
C:\mylab\book\automate_basic\send_temp\2020년도 하반기 한-EU 공동연구지원사업 신규과제 공모.zip


In [64]:
NEW_POST_LIST_XL

'C:\\mylab\\book\\automate_basic\\data\\new_post_list.xlsx'

In [59]:
df_diff.to_excel(NEW_POST_LIST_XL, encoding='cp949', index=False) # 새로운 게시물 목록을 엑셀에 저장

In [65]:
dest = BASE_DIR+"\\send_temp\\{f_name}".format(f_name='new_post_list.xlsx')
dest

'C:\\mylab\\book\\automate_basic\\send_temp\\new_post_list.xlsx'

In [66]:
from shutil import copyfile
copyfile(NEW_POST_LIST_XL, dest)

'C:\\mylab\\book\\automate_basic\\send_temp\\new_post_list.xlsx'

In [67]:
send_temp_path = BASE_DIR+'\\send_temp'
send_temp_final_path = BASE_DIR+'\\send_temp_final'

In [68]:
from datetime import datetime
f_name = datetime.now().strftime("%Y%m%d_%H%M%S") #오늘 날짜로 파일명을 만든다
email_attachment_path   = send_temp_final_path + '\\{f_name}.zip'.format(f_name=f_name)
zip(send_temp_path, email_attachment_path)

'ok'

In [69]:
emailfrom = "kusung25@gmail.com"
emailto   = "kusung25@naver.com" 
subject = "정보통신부 사업공모 게시판 신규게시물 송신 건"
email_contents_path = 'email_content.txt'

In [70]:
from email_simple import send_mail
rc = None
rc = send_mail(emailto, subject, email_contents_path, email_attachment_path)

#### 메일을 보낸 후 오늘 처리결과를 반영한다.  board_hsitory
게시물을 추가해 저장한다.

In [71]:
if rc == "ok":    
    df_hsitory = pd.read_excel(SEND_HIST_XL) 
    df_hsitory_new = pd.concat([df_diff, df_hsitory], axis=0).reset_index(drop=True)
    
    # update history
    df_hsitory_new.to_excel(SEND_HIST_XL, encoding='cp949', index=False) #encoding='cp949' 이건 뭔가?  
    mkdir_rm(send_temp_path) # 임시폴더 삭제
    mkdir_rm(send_temp_final_path)
    print("ok")

## driver.quit()

In [72]:
driver.quit()